# Implementing Recommender Systems - Lab

## Introduction

In this lab, you'll practice creating a recommender system model using `surprise`. You'll also get the chance to create a more complete recommender system pipeline to obtain the top recommendations for a specific user.


## Objectives

In this lab you will: 

- Use surprise's built-in reader class to process data to work with recommender algorithms 
- Obtain a prediction for a specific user for a particular item 
- Introduce a new user with rating to a rating matrix and make recommendations for them 
- Create a function that will return the top n recommendations for a user 


For this lab, we will be using the famous 1M movie dataset. It contains a collection of user ratings for many different movies. In the last lesson, you were exposed to working with `surprise` datasets. In this lab, you will also go through the process of reading in a dataset into the `surprise` dataset format. To begin with, load the dataset into a Pandas DataFrame. Determine which columns are necessary for your recommendation system and drop any extraneous ones.

In [1]:
import pandas as pd
df = pd.read_csv('./ml-latest-small/ratings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [2]:
# Drop unnecessary columns
new_df = df.drop('timestamp', axis=1)
new_df

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


It's now time to transform the dataset into something compatible with `surprise`. In order to do this, you're going to need `Reader` and `Dataset` classes. There's a method in `Dataset` specifically for loading dataframes.

In [3]:
from surprise import Reader, Dataset
# read in values as Surprise dataset 

reader = Reader()
data = Dataset.load_from_df(new_df,reader)


Let's look at how many users and items we have in our dataset. If using neighborhood-based methods, this will help us determine whether or not we should perform user-user or item-item similarity

In [4]:
dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  610 

Number of items:  9724


## Determine the best model 

Now, compare the different models and see which ones perform best. For consistency sake, use RMSE to evaluate models. Remember to cross-validate! Can you get a model with a higher average RMSE on test data than 0.869?

In [5]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [7]:
## Perform a gridsearch with SVD
# ⏰ This cell may take several minutes to run
params = {'n_factors': [25, 50, 100],
         'reg_all': [0.025, 0.05, 0.1],
         'lr_all': [0.0025, 0.005, 0.01],
         'n_epochs': [10, 20]}

grid_SVD = GridSearchCV(SVD,param_grid=params,n_jobs=-1)
grid_SVD.fit(data)
print(grid_SVD.best_score)
print(grid_SVD.best_params)

{'rmse': 0.8578164176433644, 'mae': 0.6577504248120454}
{'rmse': {'n_factors': 100, 'reg_all': 0.05, 'lr_all': 0.01, 'n_epochs': 20}, 'mae': {'n_factors': 100, 'reg_all': 0.05, 'lr_all': 0.01, 'n_epochs': 20}}


In [12]:
# print out optimal parameters for SVD after GridSearch

{'rmse': 0.8689250510051669, 'mae': 0.6679404366294037}
{'rmse': {'n_factors': 50, 'reg_all': 0.05}, 'mae': {'n_factors': 100, 'reg_all': 0.05}}


In [9]:
# cross validating with KNNBasic
params_KNNBasic = {'name':['cosine','pearson','MSD'], 'user_based':[False]}

grid_KNNBasic = GridSearchCV(KNNBasic, param_grid=params_KNNBasic, n_jobs=-1)
grid_KNNBasic.fit(data)
print(grid_KNNBasic.best_score)
print(grid_KNNBasic.best_params)

{'rmse': 0.9466127087281601, 'mae': 0.7255170545933074}
{'rmse': {'name': 'cosine', 'user_based': False}, 'mae': {'name': 'cosine', 'user_based': False}}


In [10]:
#ALTERNATE METHOD -- IN SOLUTION NOTEBOOK
knn_basic = KNNBasic(sim_options={'name':'pearson', 'user_based':True})
cv_knn_basic = cross_validate(knn_basic, data, n_jobs=-1)

for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([0.97267164, 0.96483807, 0.97256294, 0.97971635, 0.97044992]))
('test_mae', array([0.74945913, 0.74645346, 0.75109662, 0.75776586, 0.7489072 ]))
('fit_time', (1.114384651184082, 1.064413070678711, 1.0618948936462402, 1.0588951110839844, 1.0139226913452148))
('test_time', (1.8807368278503418, 1.8927302360534668, 1.8167734146118164, 1.924710988998413, 1.8817367553710938))
-----------------------
0.9720477864302935


In [15]:
# print out the average RMSE score for the test set

('test_rmse', array([0.97646619, 0.97270627, 0.97874535, 0.97029184, 0.96776748]))
('test_mae', array([0.75444119, 0.75251222, 0.7531242 , 0.74938542, 0.75152129]))
('fit_time', (0.46678805351257324, 0.54010009765625, 0.7059998512268066, 0.5852491855621338, 1.0139541625976562))
('test_time', (2.308177947998047, 2.4834508895874023, 2.6563329696655273, 2.652374029159546, 1.2219891548156738))
-----------------------
0.9731954260849399


In [11]:
# cross validating with KNNBaseline
params_KNNBaseline = {'name':['cosine','pearson','MSD'], 'user_based':[False]}

grid_KNNBaseline = GridSearchCV(KNNBaseline, param_grid=params_KNNBaseline, n_jobs=-1)
grid_KNNBaseline.fit(data)
print(grid_KNNBaseline.best_score)
print(grid_KNNBaseline.best_params)

{'rmse': 0.8729968597840161, 'mae': 0.6676178407460321}
{'rmse': {'name': 'cosine', 'user_based': False}, 'mae': {'name': 'cosine', 'user_based': False}}


In [13]:
# print out the average score for the test set
print(grid_KNNBaseline.best_score)

{'rmse': 0.8729968597840161, 'mae': 0.6676178407460321}


Based off these outputs, it seems like the best performing model is the SVD model with `n_factors = 50` and a regularization rate of 0.05. Use that model or if you found one that performs better, feel free to use that to make some predictions.

In [16]:
## CAH added section

from surprise.model_selection import train_test_split
from surprise import accuracy

svd = SVD(n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.05)
trainset, testset = train_test_split(data, test_size=0.2)

svd.fit(trainset)
predictions = svd.test(testset)
print(accuracy.rmse(predictions))

RMSE: 0.8706
0.8706331468993987


In [17]:
user_20_prediction = svd.predict("20","10")
user_20_prediction

Prediction(uid='20', iid='10', r_ui=None, est=3.4996962860118015, details={'was_impossible': False})

## Making Recommendations

It's important that the output for the recommendation is interpretable to people. Rather than returning the `movie_id` values, it would be far more valuable to return the actual title of the movie. As a first step, let's read in the movies to a dataframe and take a peek at what information we have about them.

In [18]:
df_movies = pd.read_csv('./ml-latest-small/movies.csv')

In [19]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Making simple predictions
Just as a reminder, let's look at how you make a prediction for an individual user and item. First, we'll fit the SVD model we had from before.

In [20]:
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(dataset)

In [21]:
svd.predict(2, 4)

Prediction(uid=2, iid=4, r_ui=None, est=3.0500290830093, details={'was_impossible': False})

In [25]:
svd.predict(2,4)[3]

3.0500290830093

This prediction value is a tuple and each of the values within it can be accessed by way of indexing. Now let's put our knowledge of recommendation systems to do something interesting: making predictions for a new user!

## Obtaining User Ratings 

It's great that we have working models and everything, but wouldn't it be nice to get to recommendations specifically tailored to your preferences? That's what we'll be doing now. The first step is to create a function that allows us to pick randomly selected movies. The function should present users with a movie and ask them to rate it. If they have not seen the movie, they should be able to skip rating it. 

The function `movie_rater()` should take as parameters: 

* `movie_df`: DataFrame - a dataframe containing the movie ids, name of movie, and genres
* `num`: int - number of ratings
* `genre`: string - a specific genre from which to draw movies

The function returns:
* rating_list : list - a collection of dictionaries in the format of {'userId': int , 'movieId': int , 'rating': float}

#### This function is optional, but fun :) 

In [26]:
def movie_rater(movie_df,num, genre=None):
    userID = 1000
    rating_list = []
    while num > 0:
        if genre:
            movie = movie_df[movie_df['genres'].str.contains(genre)].sample(1)
        else:
            movie = movie_df.sample(1)
        print(movie)
        rating = input('How do you rate this movie on a scale of 1-5, press n if you have not seen :\n')
        if rating == 'n':
            continue
        else:
            rating_one_movie = {'userId':userID,'movieId':movie['movieId'].values[0],'rating':rating}
            rating_list.append(rating_one_movie) 
            num -= 1
    return rating_list
        

In [27]:
my_ratings = movie_rater(df_movies, 5, genre='Comedy')

      movieId                                   title  genres
8754   128097  Jim Norton: American Degenerate (2013)  Comedy
How do you rate this movie on a scale of 1-5, press n if you have not seen :
n
      movieId                    title                genres
3878     5450  Lovely & Amazing (2001)  Comedy|Drama|Romance
How do you rate this movie on a scale of 1-5, press n if you have not seen :
1
      movieId                       title  genres
5897    33495  Kicking & Screaming (2005)  Comedy
How do you rate this movie on a scale of 1-5, press n if you have not seen :
3
      movieId                                           title  \
8422   111146  Alpha and Omega 3: The Great Wolf Games (2014)   

                                          genres  
8422  Action|Adventure|Animation|Children|Comedy  
How do you rate this movie on a scale of 1-5, press n if you have not seen :
n
      movieId                  title          genres
6689    58162  Run Fatboy Run (2007)  Comedy|Romance

[{'userId': 1000, 'movieId': 5450, 'rating': '1'},
 {'userId': 1000, 'movieId': 33495, 'rating': '3'},
 {'userId': 1000, 'movieId': 58162, 'rating': '1'},
 {'userId': 1000, 'movieId': 7036, 'rating': '5'},
 {'userId': 1000, 'movieId': 126548, 'rating': '4'}]

In [37]:
# try out the new function here!

      movieId                   title          genres
6579    55245  Good Luck Chuck (2007)  Comedy|Romance
How do you rate this movie on a scale of 1-5, press n if you have not seen :
5
      movieId                       title          genres
1873     2491  Simply Irresistible (1999)  Comedy|Romance
How do you rate this movie on a scale of 1-5, press n if you have not seen :
4
      movieId                  title  genres
3459     4718  American Pie 2 (2001)  Comedy
How do you rate this movie on a scale of 1-5, press n if you have not seen :
4
      movieId             title                   genres
4160     5990  Pinocchio (2002)  Children|Comedy|Fantasy
How do you rate this movie on a scale of 1-5, press n if you have not seen :
3


If you're struggling to come up with the above function, you can use this list of user ratings to complete the next segment

In [29]:
user_rating = [{'userId': 1000, 'movieId': 55245, 'rating': '5'},
 {'userId': 1000, 'movieId': 2491, 'rating': '4'},
 {'userId': 1000, 'movieId': 4718, 'rating': '4'},
 {'userId': 1000, 'movieId': 5990, 'rating': '3'}]
user_rating

[{'userId': 1000, 'movieId': 55245, 'rating': '5'},
 {'userId': 1000, 'movieId': 2491, 'rating': '4'},
 {'userId': 1000, 'movieId': 4718, 'rating': '4'},
 {'userId': 1000, 'movieId': 5990, 'rating': '3'}]

### Making Predictions With the New Ratings
Now that you have new ratings, you can use them to make predictions for this new user. The proper way this should work is:

* add the new ratings to the original ratings DataFrame, read into a `surprise` dataset 
* train a model using the new combined DataFrame
* make predictions for the user
* order those predictions from highest rated to lowest rated
* return the top n recommendations with the text of the actual movie (rather than just the index number) 

In [30]:
## add the new ratings to the original ratings DataFrame
user_ratings = pd.DataFrame(user_rating)
new_ratings_df = pd.concat([new_df, user_ratings], axis=0)
new_data = Dataset.load_from_df(new_ratings_df,reader)

In [32]:
# train a model using the new combined DataFrame
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(new_data.build_full_trainset())

In [34]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)
list_of_movies = []
for movieID in new_df['movieId'].unique():
    list_of_movies.append( (movieID, svd.predict(1000, movieID)[3]))

In [35]:
# order the predictions from highest to lowest rated

ranked_movies = sorted(list_of_movies, key=lambda x:x[1], reverse=True)
ranked_movies[:10]

[(1213, 4.540303894047375),
 (1262, 4.5346802467618605),
 (1104, 4.530946934575698),
 (1204, 4.513987416519957),
 (1089, 4.504985832716824),
 (318, 4.498675050896076),
 (898, 4.491830769009865),
 (904, 4.490977313549253),
 (858, 4.48939152742239),
 (750, 4.479613633459128)]

 For the final component of this challenge, it could be useful to create a function `recommended_movies()` that takes in the parameters:
* `user_ratings`: list - list of tuples formulated as (user_id, movie_id) (should be in order of best to worst for this individual)
* `movie_title_df`: DataFrame 
* `n`: int - number of recommended movies 

The function should use a `for` loop to print out each recommended *n* movies in order from best to worst

In [36]:
# return the top n recommendations using the 
def recommended_movies(user_ratings,movie_title_df,n):
        for idx, rec in enumerate(user_ratings):
            title = movie_title_df.loc[movie_title_df['movieId'] == int(rec[0])]['title']
            print('Recommendation # ', idx+1, ': ', title, '\n')
            n-= 1
            if n == 0:
                break
            
recommended_movies(ranked_movies,df_movies,5)

Recommendation #  1 :  914    Goodfellas (1990)
Name: title, dtype: object 

Recommendation #  2 :  961    Great Escape, The (1963)
Name: title, dtype: object 

Recommendation #  3 :  841    Streetcar Named Desire, A (1951)
Name: title, dtype: object 

Recommendation #  4 :  906    Lawrence of Arabia (1962)
Name: title, dtype: object 

Recommendation #  5 :  828    Reservoir Dogs (1992)
Name: title, dtype: object 



## Level Up (Optional)

* Try and chain all of the steps together into one function that asks users for ratings for a certain number of movies, then all of the above steps are performed to return the top $n$ recommendations
* Make a recommender system that only returns items that come from a specified genre

In [37]:
def get_recommendations_from_ratings(movie_df, n_ratings, n_recs):
    
    ratinglist = movie_rater(movie_df, n_ratings, genre=None)
    
    user_ratings = pd.DataFrame(ratinglist)
    new_ratings_df = pd.concat([movie_df, user_ratings], axis=0)
    new_data = Dataset.load_from_df(new_ratings_df,reader)
    
    svd = SVD(n_factors= 50, reg_all=0.05)
    svd.fit(new_data.build_full_trainset())
    
    list_of_movies = []
    for movieID in new_df['movieId'].unique():
        list_of_movies.append( (movieID, svd.predict(1000, movieID)[3]))
    ranked_movies = sorted(list_of_movies, key=lambda x:x[1], reverse=True)
    
    recommended_movies(ranked_movies, movie_df, n_recs)

In [38]:
get_recommendations_from_ratings(df_movies, 10, 4)

      movieId                   title                   genres
1369     1873  Misérables, Les (1998)  Crime|Drama|Romance|War
How do you rate this movie on a scale of 1-5, press n if you have not seen :
5
      movieId                                   title         genres
8267   105246  Mood Indigo (L'écume des jours) (2013)  Drama|Fantasy
How do you rate this movie on a scale of 1-5, press n if you have not seen :
2
      movieId             title           genres
1839     2446  In Dreams (1999)  Horror|Thriller
How do you rate this movie on a scale of 1-5, press n if you have not seen :
4
      movieId            title                  genres
1373     1882  Godzilla (1998)  Action|Sci-Fi|Thriller
How do you rate this movie on a scale of 1-5, press n if you have not seen :
1
      movieId                           title              genres
1525     2057  Incredible Journey, The (1963)  Adventure|Children
How do you rate this movie on a scale of 1-5, press n if you have not seen :
5
 

ValueError: too many values to unpack (expected 3)

## Summary

In this lab, you got the chance to implement a collaborative filtering model as well as retrieve recommendations from that model. You also got the opportunity to add your own recommendations to the system to get new recommendations for yourself! Next, you will learn how to use Spark to make recommender systems.